In [10]:
from gutenberg.acquire import load_etext
from gutenberg.cleanup import strip_headers
import re

In [69]:
def split_sentences(st):
    sentences = re.split(r'([.?!])\s*', st)
    sentences = [s+p for s,p in zip(sentences[::2], sentences[1::2])]
    if sentences[-1]:
        return sentences
    else:
        return sentences[:-1]
    
def extract_sentences(text):
    return split_sentences(
        ' '.join([
            re.sub(
                r'\[\d+\]', '', re.sub(r'^\s+', '', l)
            ) for l in text.split('\r\n') 
            if (l.startswith('  ') and ('*' not in l))
        ])
    )

In [33]:
sentences = []

In [34]:
# Japanese Prints  By John Gould Fletcher
text = strip_headers(load_etext(27199)).strip()
sentences += split_sentences(
    ' '.join([
        l.replace('    ', '') for l in text[text.index('_Part I_'):].split('\r\n') if l.startswith('    ')
    ])
)

In [35]:
# TS Eliot
text = strip_headers(load_etext(1567)).strip()
sentences += split_sentences(
    ' '.join([
        re.sub(r'^\s+', '', l) for l in text[899:].split('\r\n') if l.startswith('    ')
    ])
)


In [58]:
# A Child's Garden of Verses, by Robert Louis Stevenson
text = strip_headers(load_etext(19722)).strip()
sentences += extract_sentences(text[text.index('    In winter I get up at night'):])
#print(text)

In [64]:
# THE COLLECTED POEMS OF RUPERT BROOKE
text = strip_headers(load_etext(262)).strip()
sentences += extract_sentences(text[text.index('   Here in the dark, O heart;'):])
#print(text)

In [71]:
# FREEDOM, TRUTH AND BEAUTY
text = strip_headers(load_etext(20174)).strip()
sentences += extract_sentences(text[text.index('  What lineage so noble as from Sires,'):])
#print(text)

In [74]:
# LYRICS OF EARTH
text = strip_headers(load_etext(12664)).strip()
sentences += extract_sentences(text[text.index('    Mother, to whose valiant will,'):])
#print(text)

In [78]:
# Book of Sword Blades and Poppy Seed,
text = strip_headers(load_etext(1020)).strip()
sentences += extract_sentences(text[text.index('      A drifting, April, twilight sky,'):])
#print(text)

In [81]:
# Book of A Dome of Many-Coloured Glass
text = strip_headers(load_etext(261)).strip()
sentences += extract_sentences(text[text.index('          Before the Altar, bowed, he stands'):])
#print(text)

In [88]:
# A HUNDRED AND SEVENTY CHINESE POEMS
text = strip_headers(load_etext(42290)).strip()
sentences += extract_sentences(text[text.index('    “We grasp our battle-spears: we don our breast-plates of hide.'):])
#print(text)

In [91]:
# A HUNDRED AND SEVENTY CHINESE POEMS
text = strip_headers(load_etext(7889)).strip()
sentences += extract_sentences(text[text.index('  Here\'s where'):])
#print(text)

In [92]:
len(sentences)

6284

In [93]:
import re
import pandas as pd
import inflect
inflect_engine = inflect.engine()

# Load Phonemes

WORDS = {}

# Standard Dict
with open('cmudict.dict.txt', 'r') as f:
    for line in f.readlines():
        word, phonemes = line.strip().split(' ', 1)
        word = re.match(r'([^\(\)]*)(\(\d\))*', word).groups()[0]
        phonemes = phonemes.split(' ')
        syllables = sum([re.match(r'.*\d', p) is not None for p in phonemes])
        #print(word, phonemes, syllables)
        if word not in WORDS:
            WORDS[word] = []
        WORDS[word].append({
            'phonemes': phonemes,
            'syllables': syllables
        })
        
# Load custom phonemes
vowels = ['AA', 'AE', 'AH', 'AO', 'AW', 'AX', 'AXR', 'AY', 'EH', 'ER', 'EY', 'IH', 'IX', 'IY', 'OW', 'OY', 'UH', 'UW', 'UX']

with open('8659.dict.txt', 'r') as f:
    for line in f.readlines():
        word, phonemes = line.strip().split('\t', 1)
        word = re.match(r'([^\(\)]*)(\(\d\))*', word).groups()[0].lower()
        phonemes = phonemes.split(' ')
        syllables = sum([(p in vowels) for p in phonemes])
        
        if word not in WORDS:
            WORDS[word] = []
        WORDS[word].append({
            'phonemes': phonemes,
            'syllables': syllables
        })